##### Homework #1
##### Nazmul Rabbi
##### 06/21/2024

In [1]:
import json
from groq import Groq
from dotenv import load_dotenv
import os
import requests 
from tqdm.auto import tqdm

In [2]:
# load environment variables
load_dotenv()

# create client calling Groq class
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

In [3]:
print("Q1: The version.build_hash value is 42f05b9372a9a4a470db3b52817899b99a76ee73")

Q1: The version.build_hash value is 42f05b9372a9a4a470db3b52817899b99a76ee73


In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es_client = Elasticsearch('http://localhost:9200')

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
query = 'How do I execute a command in a running docker container?'

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [10]:
print("Q2: The function I used for adding the data to elastic is index")

Q2: The function I used for adding the data to elastic is index


In [11]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    # Extract the score of the top-ranking result
    top_score = response['hits']['hits'][0]['_score']

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs, top_score

In [12]:
search_results, top_score = elastic_search(query)
print("Q3: The top ranking result is", top_score)

Q3: The top ranking result is 84.050095


In [13]:
print("Q4: The 3rd question returned is", search_results[2]['question'])

Q4: The 3rd question returned is How do I copy files from a different folder into docker container’s working directory?


In [14]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"

    prompt = prompt_template.format(question=query, context=context.strip())

    return prompt

In [15]:
prompt = build_prompt(query, search_results)
print("Q5: The length of the resulting prompt is", len(prompt))

Q5: The length of the resulting prompt is 1462


In [16]:
# used a generic token estimator function because groq api doesnt support tokenzier 
def estimate_tokens(text):
    # Estimate the number of tokens based on the heuristic
    average_characters_per_token = 4.5 # 4 to 5 average characters per token
    estimated_tokens = len(text) / average_characters_per_token
    return int(estimated_tokens)

In [17]:
def llm(prompt):
    # Estimate the number of tokens in the prompt
    token_usage_estimate = estimate_tokens(prompt)

    # create a query
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    # Access token usage directly from the response object attributes
    token_usage = response.usage.total_tokens if hasattr(response, 'usage') and hasattr(response.usage, 'total_tokens') else None
    
    # print the response
    return response.choices[0].message.content, token_usage, token_usage_estimate

In [18]:
# call the groq api to get the response
response, token, estimate = llm(prompt)
print("Q6: The prompt estimated to have", estimate, "tokens and it had", token, "tokens in the actual response.")

Q6: The prompt estimated to have 324 tokens and it had 416 tokens in the actual response.


In [19]:
# LLM output
print(f"LLM response\n{response}")

LLM response
Based on the CONTEXT, to execute a command in a running docker container, you can use the following command:

```
docker exec -it <container-id> bash
```

Replace `<container-id>` with the actual ID of the container you want to execute the command in. This command will open a new shell in the container, allowing you to execute commands as if you were directly in the container.
